<a href="https://colab.research.google.com/github/Mingjie-Shen/Deeep-Learning-Homework-and-Practice/blob/main/Q3_Problem_Sheet_2_(improved_2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

import h5py
import time

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#load MNIST data
MNIST_data = h5py.File('/content/drive/My Drive/MNISTdata.hdf5', 'r')
x_train = np.float32(MNIST_data['x_train'][:] ).reshape((-1,1,28,28))
y_train = np.int32(np.array(MNIST_data['y_train'][:,0]))
x_test = np.float32( MNIST_data['x_test'][:] ).reshape((-1,1,28,28))
y_test = np.int32( np.array( MNIST_data['y_test'][:,0]  ) )

MNIST_data.close()

In [4]:
class MyCNN(nn.Module):
    def __init__(self):
        super(MyCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5,stride=2)  
        self.conv2 = nn.Conv2d(16, 64, kernel_size=5,stride=1)
        self.fc = nn.Linear(64*8*8, 10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        #x = torch.flatten(x,1)
        x = self.fc(x.reshape(-1,64*8*8))

        return F.log_softmax(x,dim=1)

In [5]:
model=MyCNN()

In [6]:
optimizer=optim.SGD(model.parameters(),lr=0.02)

In [7]:
batch_size=100
num_epochs=100
L_Y_train=len(y_train)
model.train()
train_loss=[]

In [9]:
time1=time.time()
for epoch in range(num_epochs):
  I_permutation=np.random.permutation(L_Y_train)
  x_train=x_train[I_permutation,:]
  y_train=y_train[I_permutation]
  train_accu=[]
  for i in range(0,L_Y_train,batch_size):
    x_train_batch=torch.FloatTensor(x_train[i:i+batch_size,:])
    y_train_batch=torch.LongTensor(y_train[i:i+batch_size])
    data,target=Variable(x_train_batch),Variable(y_train_batch)

    optimizer.zero_grad()

    output=model(data)
    loss=F.nll_loss(output,target)

    loss.backward()
    train_loss.append(loss.data)

    optimizer.step()
    prediction=output.data.max(1)[1]
    accuracy=(float(prediction.eq(target.data).sum())/float(batch_size))*100
    train_accu.append(accuracy)
  accuracy_epoch=np.mean(train_accu)
  print(epoch,accuracy_epoch)

time2=time.time()
delta_t=time2-time1
print(delta_t)

0 98.84833333333333
1 98.915
2 98.89833333333333
3 98.99166666666666
4 99.04833333333333
5 99.075
6 99.08833333333334
7 99.09
8 99.165
9 99.17666666666666
10 99.24
11 99.25833333333334
12 99.28
13 99.29
14 99.34833333333333
15 99.37
16 99.38333333333334
17 99.44666666666667
18 99.45666666666666
19 99.46833333333333
20 99.47666666666667
21 99.53666666666666
22 99.52
23 99.53333333333333
24 99.58166666666666
25 99.58666666666667
26 99.60833333333333
27 99.57333333333334
28 99.61
29 99.66666666666667
30 99.69333333333333
31 99.69166666666666
32 99.68833333333333
33 99.69833333333334
34 99.70166666666667
35 99.72833333333334
36 99.73833333333333
37 99.75666666666666
38 99.76
39 99.76333333333334
40 99.75833333333334
41 99.78166666666667
42 99.80333333333333
43 99.81166666666667
44 99.81
45 99.815
46 99.84
47 99.84666666666666
48 99.85
49 99.85833333333333
50 99.85833333333333
51 99.86333333333333
52 99.86833333333334
53 99.88666666666667
54 99.89333333333333
55 99.88
56 99.89166666666667
5

In [10]:
model.eval()
test_accu=[]
for i in range(0,len(y_test),batch_size):
  x_test_batch=torch.FloatTensor(x_test[i:i+batch_size,:])
  y_test_batch=torch.LongTensor(y_test[i:i+batch_size])
  data,target=Variable(x_test_batch),Variable(y_test_batch)
  optimizer.zero_grad()
  output=model(data)
  loss=F.nll_loss(output,target)
  prediction=output.data.max(1)[1]
  accuracy=(float(prediction.eq(target.data).sum())/float(batch_size)*100)
  test_accu.append(accuracy)

accuracy_test=np.mean(test_accu)
print('The accuracy on the test set is',accuracy_test)

The accuracy on the test set is 98.77
